#### spateGAN-ERA5 spatiotemporal precipitation downscaling example
https://doi.org/10.48550/arXiv.2411.16098

In [1]:
%load_ext autoreload
%autoreload 2
import torch
from model import Generator  
from train import train, validation
from data import readFile, make_temporal_batches, block_average_pooling
import xarray as xr
import numpy as np
from einops import rearrange

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


#### load model and weights and run the model


#### AORC to MRMS

In [4]:
mrms = readFile('data/mrms/mrms_nystate_*.nc', 'mrms', 720, 105, 155)
mrms = mrms.reshape(-1, 144, 5, 105, 155).mean(axis=2)
mrms = mrms.reshape(-1, 6, 105, 155)
mean_val = np.nanmean(mrms)
mrms = np.where(np.isnan(mrms), 0.0, mrms)
print(mrms.shape)

data_aorc = readFile('data/aorc_11/APCP_surface_*.nc', 'APCP_surface', 24, 21, 31)
data_aorc = data_aorc.reshape(-1, 1, 21, 31)[:, :, ::-1, :]
mean_val = np.nanmean(data_aorc)
data_aorc = np.where(np.isnan(data_aorc), 0.0, data_aorc)
print(data_aorc.shape)
input, target = make_temporal_batches(data_aorc, mrms)
print(input.shape)
print(target.shape)

final shape (20, 720, 105, 155)
(480, 6, 105, 155)
final shape (20, 24, 21, 31)
(480, 1, 21, 31)
(96, 1, 5, 21, 31)
(96, 1, 30, 105, 155)


In [ ]:
train(input, target)